## Loading cell positions and colors from whole worm data

In [1]:
import csv

%matplotlib inline  
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['legend.handlelength'] = 0
plt.rcParams["figure.figsize"] = (16,3)

def translate_pal_color(r_z,g_z,b_z):
    # Very, very, very approximately!
    def z_to_0_1(z):
        return min(1,max(0,(z)/5.))
    rgb = (z_to_0_1(r_z),z_to_0_1(g_z),z_to_0_1(b_z))
    #print('Converted expression (%s,%s,%s) to %s'%(r_z,g_z,b_z,rgb))
    return rgb

saved_colors = {}
def load_positions(csv_file):
    positions = {}
    file_name = csv_file.split('/')[-1]
    if not file_name in saved_colors:
        saved_colors[file_name] = {}

    with open(csv_file, newline='') as csvfile:
        linereader = csv.reader(csvfile, delimiter=',')
        for row in linereader:
            index_id = 1 if len(row)==5 else 0
            index_x = 2 if len(row)==5 else 1
            index_y = 3 if len(row)==5 else 2
            index_z = 4 if len(row)==5 else 3
            cell_name = row[index_id]
            #print(cell_name)
            
            if cell_name != 'User ID':
                positions[cell_name] = {}
                positions[cell_name]['ap_position'] = float(row[index_x])
                positions[cell_name]['dv_position'] = float(row[index_y])
                positions[cell_name]['lr_position'] = float(row[index_z])

                positions[cell_name]['color'] = 'b'
                if len(row)>5:
                    positions[cell_name]['color'] = translate_pal_color(float(row[5]),
                                                                        float(row[6]),
                                                                        float(row[7]))
                    saved_colors[file_name][cell_name] = positions[cell_name]['color']
                elif cell_name in saved_colors[file_name]:
                    positions[cell_name]['color'] = saved_colors[file_name][cell_name]

    print('Loaded in %i positions from %s'%(len(positions),csv_file))
    return positions

In [2]:

def generate_ap_dv_plot(positions, ref, type):

    plt.figure()
    plt.title('AP-DV - %s (%s.csv)'%(type, ref))
    plt.axis('equal')

    for neuron in sorted(positions.keys()):
    
        label='%s'%neuron
        plt.plot(positions[neuron]['ap_position'], 
                 positions[neuron]['dv_position'], 
                 marker="o",
                 linestyle=None,
                 color=positions[neuron]['color'],
                 label=label)
        
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=15)

def generate_ap_lr_plot(positions, ref, type):
        
    plt.figure()
    plt.title('AP-LR - %s (%s.csv)'%(type, ref))
    plt.axis('equal')

    for neuron in positions:

        label=None
        plt.plot(positions[neuron]['ap_position'], 
                 positions[neuron]['lr_position'], 
                 marker="o",
                 linestyle=None,
                 color=positions[neuron]['color'],
                 label=label)

In [ ]:
positions = load_positions('data/WholeWorm/atlas.csv')
generate_ap_dv_plot(positions, 'atlas', 'Averaged atlas(?)')
generate_ap_lr_plot(positions, 'atlas', 'Averaged atlas(?)')

Loaded in 300 positions from data/WholeWorm/atlas.csv


In [ ]:
worms = {}
all_positions = {}
with open('data/WholeWorm/naming.txt') as f:
    lines = f.readlines()
    for l in lines:
        w = l.split('=')
        worms[w[0].strip()] = w[1].strip()
print(worms)

        
for ref in worms:
    filename = 'data/WholeWorm/Raw Unstraightened Individuals with colors/%s.csv'%ref
    positions = load_positions(filename)
    all_positions[ref] = positions
    generate_ap_dv_plot(positions, ref, worms[ref])
    

In [ ]:
for ref in worms:
    filename = 'data/WholeWorm/Straightened Individuals/%s.csv'%ref
    positions = load_positions(filename)
    all_positions['%s_st'%ref] = positions
    generate_ap_dv_plot(positions, ref, worms[ref])

In [ ]:
for ref in all_positions:
    print('Position set: %s, %i neurons'%(ref, len(all_positions[ref])))
    pos = all_positions[ref]
    for neuron in list(pos.keys())[:2]:
        print('   %s: (%s,%s,%s)'%(neuron, pos[neuron]['ap_position'], pos[neuron]['dv_position'], pos[neuron]['lr_position']))

In [ ]:
import neuroml
from pyneuroml import pynml

import neuroml.writers as writers

o = 200
o_st = 150

offset = {'1_YAw':(0,0,0),'1_YAw_st':(0,0,0)
         ,'2_AMw':(0,0,o),'2_AMw_st':(0,0,o_st)
         ,'3_NPv16-64_YAw':(0,0,o*2),'3_NPv16-64_YAw_st':(0,0,o_st*2)
         ,'7_YAw':(0,0,o*3),'7_YAw_st':(0,0,o_st*3)
         ,'9_YAw':(0,0,o*4),'9_YAw_st':(0,0,o_st*4)
         ,'14_Aw':(0,0,o*5),'14_Aw_st':(0,0,o_st*5)
         ,'24_L4w':(0,0,o*6),'24_L4w_st':(0,0,o_st*6)}

transform_x = {'1_YAw':1}
transform_y = {'1_YAw':1}

sets_to_include = offset.keys()
for s in sets_to_include:
    transform_x[s]=1
    transform_y[s]=1

prototype_neuron = 'GenericNeuronCell'
cell = neuroml.FitzHughNagumoCell(id=prototype_neuron, I=0.1)

nml_doc_all = neuroml.NeuroMLDocument(id="NeuroPAL_All")
nml_doc_all.fitz_hugh_nagumo_cells.append(cell)

net_all = neuroml.Network(id=nml_doc_all.id)
nml_doc_all.networks.append(net_all)

nml_doc_all_st = neuroml.NeuroMLDocument(id="NeuroPAL_All_straightened")
nml_doc_all_st.fitz_hugh_nagumo_cells.append(cell)

net_all_st = neuroml.Network(id=nml_doc_all_st.id)
nml_doc_all_st.networks.append(net_all_st)

for ref in sets_to_include:
    print(' - Adding positions for %s'%ref)
    ref_ = ref.replace(' ','_').replace('-','_')
    nml_doc = neuroml.NeuroMLDocument(id="NeuroPAL_%s"%ref_)
    nml_doc.fitz_hugh_nagumo_cells.append(cell)

    net = neuroml.Network(id=nml_doc.id)
    nml_doc.networks.append(net)

    for neuron in all_positions[ref]:
        
        if not neuron=='0':

            # Add in file for individual part
            pop = neuroml.Population(id="pop_%s"%neuron, component=prototype_neuron, type="populationList")
            net.populations.append(pop)
            pos = all_positions[ref]
            c = pos[neuron]['color']
            #print('Adding neuron %s, color: %s'%(neuron,c))
            pop.properties.append(neuroml.Property(tag="color", value='%f %f %f'%(c[0],c[1],c[2])))
            pop.properties.append(neuroml.Property(tag="radius", value=2))

            inst = neuroml.Instance(id="0")
            pop.instances.append(inst)

            inst.location = neuroml.Location(x=pos[neuron]['ap_position'], 
                                     y=pos[neuron]['dv_position'], 
                                     z=pos[neuron]['lr_position'])
        
            
            # Add in file for all parts
            pop = neuroml.Population(id="%s_%s"%(neuron,ref_), component=prototype_neuron, type="populationList")
            
            if '_st' in ref:
                net_all_st.populations.append(pop)
            else:
                net_all.populations.append(pop)
            
            c = pos[neuron]['color']
            pop.properties.append(neuroml.Property(tag="color", value='%f %f %f'%(c[0],c[1],c[2])))
            pop.properties.append(neuroml.Property(tag="radius", value=2))

            inst = neuroml.Instance(id="0")
            pop.instances.append(inst)
            

            inst.location = neuroml.Location(
                               x=transform_x[ref]*pos[neuron]['ap_position']+offset[ref][0], 
                               y=transform_y[ref]*pos[neuron]['dv_position']+offset[ref][1], 
                               z=pos[neuron]['lr_position']+offset[ref][2])
            
            


    ###### Write the NeuroML network file ######   
    nml_file = 'NeuroML2/%s.net.nml'%net.id
    writers.NeuroMLWriter.write(nml_doc, nml_file)
    print("Written network file to: "+nml_file)

    ###### Validate the NeuroML ######    
    from neuroml.utils import validate_neuroml2
    validate_neuroml2(nml_file)


    ###### Convert NeuroML network to png and display ######    
    #pynml.nml2_to_svg(nml_file)
        
            

###### Write the NeuroML network file with all parts ######   
nml_file_all = 'NeuroML2/%s.net.nml'%net_all.id
writers.NeuroMLWriter.write(nml_doc_all, nml_file_all)
print("Written network file to: "+nml_file_all)

###### Validate the NeuroML ######    
from neuroml.utils import validate_neuroml2
validate_neuroml2(nml_file_all)

###### Write the NeuroML network file with all parts - straightened ######   
nml_file_all_st = 'NeuroML2/%s.net.nml'%net_all_st.id
writers.NeuroMLWriter.write(nml_doc_all_st, nml_file_all_st)
print("Written network file to: "+nml_file_all_st)

###### Validate the NeuroML ######    
from neuroml.utils import validate_neuroml2
validate_neuroml2(nml_file_all_st)


###### Convert NeuroML network to png and display ######    
#pynml.nml2_to_svg(nml_file_hm)
        
            